# 80 / 20

In [ ]:
import pickle as p
import numpy as np
import pandas as pd

# load the above and below 80th percentile split data
with open('Pickles/80_20_split.pkl', 'rb') as file:
    split80_20 = p.load(file)

In [ ]:
split80_20.keys()

In [ ]:
import pandas as pd

# load the goal information
goals = pd.read_excel('Tore.xlsx')
goals.head(3)

In [ ]:
# scoring_ha = which team scored

goals['time_string'] = goals['Timestamp_point'].apply(str)
goals.loc[goals['Scoring Team'] == goals['Home'], 'Scoring_ha'] = -3
goals.loc[goals['Scoring Team'] == goals['Away'], 'Scoring_ha'] = -1
goals.to_excel("goal_table.xlsx")
goals

#df1.to_excel("output.xlsx")  

## Mean success score for goals and no goals
### We take all 45 goals as goals for 30m intervals and only the 39 within the box for 16m

In [ ]:
from statistics import mean
import pandas as pd
import pickle as p

# load complete data
with open('Pickles/AllDatapoints.pkl', 'rb') as file:
    full_dict = p.load(file)
    
full_dict.keys()
datasets = list(full_dict.keys())                            # datasets
goal_dict = {}
df_SS_goal = pd.DataFrame(index=['Goal', 'No_goal'])

for dataset in datasets:
    goal_dict["_".join([dataset, 'GOAL'])] = []                  # for every previous dataset in my dictionary i now want two: 
    goal_dict["_".join([dataset, 'NOgoal'])] = [] 
    for frame in range(len(full_dict[dataset])):                # for every frame
        if dataset.startswith('30'):                                                # for the 30m intervals we don't need to check whether the goal was scored from within the penalty area
            if float(full_dict[dataset][frame][6]) in goals.Timestamp_point.values:        # to identify goals the time has to match
                index_list = np.where(goals["Timestamp_point"].values == float(full_dict[dataset][frame][6]))      # all those which match by the time stamp
                for i in index_list:
                    if goals.Gamecode[i[0]] in full_dict[dataset][frame][2] and goals.Scoring_ha[i[0]] == full_dict[dataset][frame][1]:       # have to also match by the match id and the team (scoring team)
                        goal_dict["_".join([dataset, 'GOAL'])].append(full_dict[dataset][frame][0])         # if yes we append to the goal data set
                        # print(split80_20[dataset][frame])
                        # print(dataset)
                    else:
                        goal_dict["_".join([dataset, 'NOgoal'])].append(full_dict[dataset][frame][0])      # else we append to the no goal data set
            else: 
                goal_dict["_".join([dataset, 'NOgoal'])].append(full_dict[dataset][frame][0])              # else we append to the no goal data set
        elif dataset.startswith('16'):                                               # for 16m intervals we have to check if scored from within the penalty are
            if float(full_dict[dataset][frame][6]) in goals.Timestamp_point[goals.David==1].values:        # to identify goals the time has to match (==1 --> from within the box)
                index_list = np.where(goals["Timestamp_point"].values == float(full_dict[dataset][frame][6]))      # all those which match by the time stamp
                for i in index_list:
                    if goals.Gamecode[i[0]] in full_dict[dataset][frame][2] and goals.Scoring_ha[i[0]] == full_dict[dataset][frame][1]:       # have to also match by the match id and the team (scoring team)
                        goal_dict["_".join([dataset, 'GOAL'])].append(full_dict[dataset][frame][0])         # if yes we append to the goal data set
                        # print(split80_20[dataset][frame])
                        # print(dataset)
                    else:
                        goal_dict["_".join([dataset, 'NOgoal'])].append(full_dict[dataset][frame][0])      # else we append to the no goal data set
            else: 
                goal_dict["_".join([dataset, 'NOgoal'])].append(full_dict[dataset][frame][0])              # else we append to the no goal data set

    print(dataset)
    print(mean(goal_dict["_".join([dataset, 'GOAL'])]))
    print(mean(goal_dict["_".join([dataset, 'NOgoal'])]))
    df_SS_goal[dataset] = [mean(goal_dict["_".join([dataset, 'GOAL'])]), mean(goal_dict["_".join([dataset, 'NOgoal'])])]
    df_SS_goal

In [ ]:
df_SS_goal.to_csv('Results/Success_Score_goal.csv')
df_SS_goal

In [ ]:
df_SS_perc = pd.DataFrame(index=['Goal', 'No_goal'])
for dataset in datasets:
    perc_mean = []
    
    for v in range(0,2):
        perc = []
        i = 0
        for line in full_dict[dataset]:
            i += 1
            if line[0] > (df_SS_goal[dataset][v]-0.1) and line[0] < (df_SS_goal[dataset][v]+0.1):
                print(dataset)
                print(line)
                print(len(full_dict[dataset]))
                print(i/len(full_dict[dataset]))
                print(i)
                print('END OF THIS BLOCK')
                print('')
                perc.append(i/len(full_dict[dataset]))
        perc_mean.append(mean(perc))
    df_SS_perc[dataset] = perc_mean
    

In [ ]:
df_SS_perc.to_csv('Results/Success_Score_percentile.csv')
df_SS_perc

In [ ]:
#need a column to which subset each frame /SS belongs
# so that i can then do what MS did and split goals scored on a frequent SS in different groups

import pandas as pd

with open('Pickles/AllDatapoints.pkl', 'rb') as file:            # import the dictionary where data is not split by group or 80 percentile
    full_dict = p.load(file)

with open('Pickles/index_limits_80.pkl', 'rb') as file:                 # import the boarders /limits (index) for over under 80
    limits_80 = p.load(file)

with open('Pickles/index_limits_8.pkl', 'rb') as file:                 # import the boarders /limits (index) for the 8 groups
    limits_8 = p.load(file)
    
success_scores = {}                                      # create a dictionary to sort the success scores

datasets = list(full_dict.keys())                    # for all datasets by area & interval (not split)  
for dataset in datasets:
    success_scores[dataset] = []                                   # create a success score vector 
    for frame in range(len(full_dict[dataset])):               # for every frame (by number, not the frame itself)     
        success_scores[dataset].append(full_dict[dataset][frame][0])     # append the success score to the vector
     
    
# by index determine wether a success score falls in under or over the 80th percentile

for dataset in datasets:
    success_scores["_".join([dataset, 'ou'])] = np.repeat("NaN", limits_80[dataset][1])
    success_scores["_".join([dataset, 'ou'])][:limits_80[dataset][0]] = 'U'    # up to the limit it is under = U
    success_scores["_".join([dataset, 'ou'])][limits_80[dataset][0]:] = 'O'    # Upwards of the limit it is over = O


## by index determine the group a success score falls into (1-8)

for dataset in datasets:
    success_scores["_".join([dataset, "8"])] = np.repeat("NaN", limits_80[dataset][1])     # first create a vector with NaN (full length)
    for i in range(len(limits_8[dataset])):                                               # fill it based on the read limits (s.above)
        if i == 0:
            success_scores["_".join([dataset, "8"])][0:int(limits_8[dataset][i])] = i + 1    # between 0 and boarder of group 1
        else:
            success_scores["_".join([dataset, "8"])][int(limits_8[dataset][i-1]):int(limits_8[dataset][i])] = i + 1  # between boarder of group -1 and group
            


In [ ]:
success_scores.keys()

In [ ]:
success_scores['16m_0010s_ou']

In [ ]:
success_scores['16m_0010s_8']

In [ ]:
# sorted (ascending)
success_scores['16m_0010s']

# Goals over and under 80th percentile

In [ ]:
goal_count = {}                                 # new dictionary to count the goals
datasets = list(full_dict.keys())           # datasets in this case not split based on percentiles (area & interval)

with open('Pickles/Boarder_values_10.pkl', 'rb') as file:             # boarder values for the 80th percentile (the 8th element in each array)
    percentiles_10 = p.load(file)
    

for dataset in datasets:
    goal_count["_".join([dataset, 'over80'])] = 0                    # for every dataset i count goals over
    goal_count["_".join([dataset, 'under80'])] = 0                   # and under --> 2 lists in dictionary
    for frame in range(len(full_dict[dataset])):                   # every frame is checked
        if dataset.startswith('30'):
            if float(full_dict[dataset][frame][6]) in goals.Timestamp_point.values:        #wether the time fits one of the goals
                index_list = np.where(goals["Timestamp_point"].values == float(full_dict[dataset][frame][6]))    # if yes we get the index of the goal
                for i in index_list:
                    #print(goal_count["_".join([dataset, 'under80'])])
                    #print(goal_count["_".join([dataset, 'over80'])])
                    if goals.Gamecode[i[0]] in full_dict[dataset][frame][2] and goals.Scoring_ha[i[0]] == full_dict[dataset][frame][1]:    # and check whether matchcode and scoring team fit as well --> confirm it is a goal frame
                        if full_dict[dataset][frame][0] < percentiles_10[dataset][8]:                                               # if under the boarder value
                            goal_count["_".join([dataset, 'under80'])] = float(goal_count["_".join([dataset, 'under80'])]) + 1 # we add + 1 to under count
                        elif full_dict[dataset][frame][0] > percentiles_10[dataset][8]:                                             # if over the boarder value
                            goal_count["_".join([dataset, 'over80'])] = float(goal_count["_".join([dataset, 'over80'])]) + 1   # we add +1  to over count
                        elif full_dict[dataset][frame][0] == percentiles_10[dataset][8]:                                            # if excatly the boarder value
                            tot_frequency = len(np.where(np.array(success_scores[dataset]) == full_dict[dataset][frame][0])[0])         # freqeuncy of frames with that success score
                            over_frequency = len(np.where((np.array(success_scores[dataset]) == full_dict[dataset][frame][0]) & (np.array(success_scores["_".join([dataset, "ou"])]) == 'O'))[0])         # freqeuncy of frames with that success score in over
                            under_frequency = len(np.where((np.array(success_scores[dataset]) == full_dict[dataset][frame][0]) & (np.array(success_scores["_".join([dataset, "ou"])]) == 'U'))[0])        # freqeuncy of frames with that success score in under
                            o_goals = over_frequency / tot_frequency                                                              # goal share for over 
                            u_goals = under_frequency / tot_frequency                                                             # goal share for under
                            goal_count["_".join([dataset, 'under80'])] = float(goal_count["_".join([dataset, 'under80'])]) + u_goals    # add the share
                            #print(goal_count["_".join([dataset, 'under80'])])
                            goal_count["_".join([dataset, 'over80'])] = float(goal_count["_".join([dataset, 'over80'])]) + o_goals      # add the share
        elif dataset.startswith('16'):
            if float(full_dict[dataset][frame][6]) in goals.Timestamp_point[goals.David == 1].values:        #whether the time fits one of the goals (only the 39 in the box ==1)
                index_list = np.where(goals["Timestamp_point"].values == float(full_dict[dataset][frame][6]))    # if yes we get the index of the goal
                for i in index_list:
                    #print(goal_count["_".join([dataset, 'under80'])])
                    #print(goal_count["_".join([dataset, 'over80'])])
                    if goals.Gamecode[i[0]] in full_dict[dataset][frame][2] and goals.Scoring_ha[i[0]] == full_dict[dataset][frame][1]:    # and check whether matchcode and scoring team fit as well --> confirm it is a goal frame
                        if full_dict[dataset][frame][0] < percentiles_10[dataset][8]:                                               # if under the boarder value
                            goal_count["_".join([dataset, 'under80'])] = float(goal_count["_".join([dataset, 'under80'])]) + 1 # we add + 1 to under count
                        elif full_dict[dataset][frame][0] > percentiles_10[dataset][8]:                                             # if over the boarder value
                            goal_count["_".join([dataset, 'over80'])] = float(goal_count["_".join([dataset, 'over80'])]) + 1   # we add +1  to over count
                        elif full_dict[dataset][frame][0] == percentiles_10[dataset][8]:                                            # if excatly the boarder value
                            tot_frequency = len(np.where(np.array(success_scores[dataset]) == full_dict[dataset][frame][0])[0])         # freqeuncy of frames with that success score
                            over_frequency = len(np.where((np.array(success_scores[dataset]) == full_dict[dataset][frame][0]) & (np.array(success_scores["_".join([dataset, "ou"])]) == 'O'))[0])         # freqeuncy of frames with that success score in over
                            under_frequency = len(np.where((np.array(success_scores[dataset]) == full_dict[dataset][frame][0]) & (np.array(success_scores["_".join([dataset, "ou"])]) == 'U'))[0])        # freqeuncy of frames with that success score in under
                            o_goals = over_frequency / tot_frequency                                                              # goal share for over 
                            u_goals = under_frequency / tot_frequency                                                             # goal share for under
                            goal_count["_".join([dataset, 'under80'])] = float(goal_count["_".join([dataset, 'under80'])]) + u_goals    # add the share
                            #print(goal_count["_".join([dataset, 'under80'])])
                            goal_count["_".join([dataset, 'over80'])] = float(goal_count["_".join([dataset, 'over80'])]) + o_goals      # add the share

                        
# create a dataframe storing the count of goals in a practical format and save to csv
goal_count_80_df = pd.DataFrame() 


stri = ['over80', 'under80']
strin = ['over', 'under']
for dataset in datasets:
    counts = []
    for i in range(len(stri)):
        counts.append(goal_count["_".join([dataset, stri[i]])])
        counts.append(len(split80_20["_".join([dataset, strin[i]])]))        # add the length of the data for chi square
    counts.append(len(success_scores[dataset]))
    goal_count_80_df[dataset] = counts
    


goal_count_80_df.to_csv('Results/goal_Count_80.csv')
goal_count_80_df

# Goals in 8 groups

In [ ]:
# load the 8 groups split data
with open('Pickles/8_split.pkl', 'rb') as file:
    split_8 = p.load(file)

In [ ]:
split_8.keys()

In [ ]:
limits_8

In [ ]:
goal_count_8 = {}
datasets = list(full_dict.keys())


with open('Pickles/Boarder_values_8.pkl', 'rb') as file:
    boarder_values_8 = p.load(file)
    
groups =list(range(1,9))

tot_frequency_dict = {}
group_frequencies_dict = {}
goal_share = {}

for dataset in datasets:                                   # for each combination of area & interval
    for group in groups:                                   # for each group (by percentile) --> 1 to 8
        goal_count_8["_".join([dataset, str(group)])] = 0  # empty list in the dictionary for every group for every dataset
        tot_frequency_dict["_".join([dataset, str(group)])] = []         # dictionary for the total frequency of that success score
        group_frequencies_dict["_".join([dataset, str(group)])] = []     # dictionary for the freqeuncy of that success score in that group
        goal_share["_".join([dataset, str(group)])] = []                 # dictionary for the share of the goals scored at that success score for each group
    for frame in range(len(full_dict[dataset])):       # for every frame (loop by number; not the frame itself)
        if (float(full_dict[dataset][frame][6]) in goals.Timestamp_point.values and dataset.startswith('30')) or (float(full_dict[dataset][frame][6]) in goals.Timestamp_point[goals.David==1].values):             # is the time in the frame in the list of goals (-->potential goal frame)
            # either a 30m datapoint with the right time stamp or a 16m datapoint with the right time stamp matching only the 39
            index_list = np.where(goals["Timestamp_point"].values == float(full_dict[dataset][frame][6]))      # create a list the index of the goal in the goal data frame
            for i in index_list:                                                                # for every goal with that time
                if goals.Gamecode[i[0]] in full_dict[dataset][frame][2] and goals.Scoring_ha[i[0]] == full_dict[dataset][frame][1]:  # is the gamecode identical with the frame and the scoring team the correct one
                    for group in groups:                   # for each group (by percentile) --> 1 to 8
                        if group == 1:                                                   # if group is = 1 group-1 is not usable for lower limit so we here have to use 0 - group1 
                            if full_dict[dataset][frame][0] < boarder_values_8[dataset][group-1]:                   # is the success score smaller than the limit of group1
                                goal_count_8["_".join([dataset, str(group)])] = goal_count_8["_".join([dataset, str(group)])] + 1    # if yes it counts as a full goal to goals in group 
                            elif full_dict[dataset][frame][0] == boarder_values_8[dataset][group-1]:   # if not we check if it is exactly the boarder value; if yes we create a share based on the frequency of that success score in each group
                                tot_frequency_dict["_".join([dataset, str(group)])] = len(np.where(np.array(success_scores[dataset]) == full_dict[dataset][frame][0])[0])   # the tot_frequency is frequency of that success score across all groups
                                #print(tot_frequency_dict["_".join([dataset, str(group)])])
                                group_frequencies_dict["_".join([dataset, str(group)])] = len(np.where((np.array(success_scores[dataset]) == full_dict[dataset][frame][0]) & (np.array(success_scores["_".join([dataset, "8"])]) == str(group)))[0])  # the freqeuncy of that success score in this group
                                #print(group_frequencies_dict["_".join([dataset, str(group)])])
                                goal_share["_".join([dataset, str(group)])] = group_frequencies_dict["_".join([dataset, str(group)])]/tot_frequency_dict["_".join([dataset, str(group)])]   # goal share = frequency in that group / overall frequency (* goals with that success score)
                                #print(goal_count_8["_".join([dataset, str(group)])])
                                goal_count_8["_".join([dataset, str(group)])] = float(goal_count_8["_".join([dataset, str(group)])]) + goal_share["_".join([dataset, str(group)])]      # finally we add that share to that group
                                #print(goal_share["_".join([dataset, str(group)])])
                                #print(goal_count_8["_".join([dataset, str(group)])])
                        else:
                            if full_dict[dataset][frame][0] < boarder_values_8[dataset][group-1] and full_dict[dataset][frame][0] > boarder_values_8[dataset][group-2]: # if group is > 1 we can just check with limits (group -1 upper limit to group upper limit)
                                goal_count_8["_".join([dataset, str(group)])] = goal_count_8["_".join([dataset, str(group)])] + 1                   # if yes it counts as a full goal to goals in group 
                            elif (full_dict[dataset][frame][0] == boarder_values_8[dataset][group-1]) | (full_dict[dataset][frame][0] == boarder_values_8[dataset][group-2]):                      # if not we check if it is exactly the boarder value; if yes we create a share based on the frequency of that success score in each group
                                tot_frequency_dict["_".join([dataset, str(group)])] = len(np.where(np.array(success_scores[dataset]) == full_dict[dataset][frame][0])[0])                           # the tot_frequency is frequency of that success score across all groups
                                group_frequencies_dict["_".join([dataset, str(group)])] = len(np.where((np.array(success_scores[dataset]) == full_dict[dataset][frame][0]) & (success_scores["_".join([dataset, "8"])] == str(group)))[0]) # the tot_frequency is frequency of that success score across all groups
                                goal_share["_".join([dataset, str(group)])]= group_frequencies_dict["_".join([dataset, str(group)])]/tot_frequency_dict["_".join([dataset, str(group)])]           # goal share = frequency in that group / overall frequency (* goals with that success score)
                                goal_count_8["_".join([dataset, str(group)])] = float(goal_count_8["_".join([dataset, str(group)])]) + goal_share["_".join([dataset, str(group)])]                # finally we add that share to that group                                   
                                #print(goal_count_8["_".join([dataset, str(group)])])

                                
# create a dataframe storing the count of goals in a practical format and save to csv
goal_count_8_df = pd.DataFrame() 

for dataset in datasets:
    counts = []
    for i in list(range(1,9)):
        counts.append(goal_count_8["_".join([dataset, str(i)])])
    goal_count_8_df[dataset] = counts
    


goal_count_8_df.to_csv('Results/goal_Count_8.csv')
goal_count_8_df